In [1]:
import pandas as pd
import numpy as np
import random
import datetime as dt
import psycopg2 as db
from sqlalchemy import create_engine

# Импорт парметров подключения к БД
from config import database

#Подключение к БД
conn = db.connect(**database)
cursor = conn.cursor()

engine = create_engine('postgresql+psycopg2://{user}:{password}@{host}:5432/{database}'.format(**database))

# Функция для загрузки таблицы с информацией об АЦ
def add_tanker_information_to_db(conn):
    
    query = "select * from superset.kind_tanker_v"

    df_tanker = pd.read_sql_query(query, conn)
    df_tanker.index = df_tanker.loc[:, 'tanker_name']
    df_tanker = df_tanker[['petrol1', 'diesel', 'petrol2', 'petrol3']]
    
    conn.commit()

    return df_tanker

#Применение функции
df_tanker = add_tanker_information_to_db(conn)

# Функция загрузки с БД таблицу с НСИ по всем резервуарам
def get_rez_table(conn):

    query = '''select pnpo_id as pnpo, azs_id as azs, rez_id as rez, rez_volume,
                rez_ref.oil_type as oil_id,
                oil_ref.name as oil_type, tanker_ref.name as kind_tanker
                from superset.rez_ref  
                left join superset.tanker_ref on tanker_ref.id_tanker=rez_ref.kind_tanker
                left join superset.oil_ref on oil_ref.id_oil=rez_ref.oil_type
                ORDER BY azs_id ASC'''

    df_with_rez = pd.read_sql_query(query, conn)
    
    conn.commit()

    return df_with_rez

# Применение функции 
df_with_rez = get_rez_table(conn)

# Функция для загрузки таблицы с информацией о пистолетов
def get_pump_df(conn):
    
    query = "select * from superset.pump_ref order by pump_id"

    df_pump = pd.read_sql_query(query, conn)
    
    conn.commit()

    return df_pump

#Применение функции
df_pump = get_pump_df(conn)

# Функция для загрузки таблицы с информацией о сменах
def get_shift_df(conn):
    
    query = "select * from superset.shift_ref order by shift_id"

    df_shift = pd.read_sql_query(query, conn)
    
    conn.commit()

    return df_shift

#Применение функции
df_shift = get_shift_df(conn)

# Функция для загрузки таблицы с информацией о расписаниях по сменам
def get_schedle_df(conn):
    
    query = "select * from superset.schedle_shift_ref order by schedle_shift_id"

    df_schedle = pd.read_sql_query(query, conn)
    
    conn.commit()

    return df_schedle

#Применение функции
df_schedle = get_schedle_df(conn)

#Применение функцииОбъединение двух таблиц по сменам
df_shift_with_schedule = df_shift.merge(df_schedle, how='left', left_on='azs_id', right_on='azs_id')

#Инициализация вводимых пользователем парметров
print('Введите начало периода генерации данных в формате %d-%m-%Y %H:%M:%S')
date_start = str(input())     #'01-05-2022 12:00:00' #VARRIBLE
print('Введите окончание периода генерации данных в формате %d-%m-%Y %H:%M:%S')
date_finish = str(input()) #VARRIBLE '06-05-2022 12:00:00'
print('Введите общий процент дебалансов на одной АЗС')
mutual_part_for_noise = 100#int(input()) 
mutual_part_for_noise = mutual_part_for_noise*0.01 # Variable
print('Введите процент положительных дебалансов среди общих на одной АЗС')
positive_part_for_noise = 50 #int(input()) 
positive_part_for_noise = positive_part_for_noise*0.01 # Variable
print('Введите левую границу относительного отклонения положительных дебалансов')
left_positive_border = 0#int(input()) 
left_positive_border = left_positive_border*0.01 # Variable
print('Введите правую границу относительного отклонения положительных дебалансов')
right_positive_border = 10#int(input()) 
right_positive_border = right_positive_border*0.01 # Variable
print('Введите левую границу относительного отклонения отрицательных дебалансов')
left_negative_border = 0#int(input()) 
left_negative_border = left_negative_border*0.01 # Variable
print('Введите правую границу относительного отклонения отрицательных дебалансов')
right_negative_border = 10#int(input()) 
right_negative_border = right_negative_border*0.01 # Variable
print('Для какого количества АЗС Вы хотите сгенерировать данные')
count_of_azs = int(input()) #

# Здесь нужен цилк перебирающий все АЗС
for azs_name in range(1, count_of_azs+1): #df_with_rez['azs'].unique():#

    # Функция для перевода даты и фильтрация таблицы по АЗС
    def prepare_start_data(azs_name, date_start, date_finish, df_shift_with_schedule, df_pump):

        current_date = dt.datetime.strptime(date_start, '%d-%m-%Y %H:%M:%S')
        finish_date = dt.datetime.strptime(date_finish, '%d-%m-%Y %H:%M:%S')
        df_for_count_data = df_with_rez[df_with_rez['azs']==azs_name].reset_index(drop=True)
        df_for_check_data = df_pump[df_pump['azs_id']==azs_name].reset_index(drop=True)
        df_for_shift_data = df_shift_with_schedule[df_shift_with_schedule['azs_id']==azs_name].reset_index(drop=True)
        return current_date, finish_date, df_for_count_data, df_for_check_data, df_for_shift_data

    #Применение функции
    current_date, finish_date, df_for_count_data, df_for_check_data, df_for_shift_data = prepare_start_data(azs_name, date_start, date_finish, df_shift_with_schedule, df_pump)

    #Создание словарика для периодичности слива из Резервуара
    def create_dict_for_data(df_for_count_data):
        x = len(df_for_count_data)
        x1 = 520
        x2 = 259
        x3 = 1309
        dict_for_data = {'25000':x1, '50000':x2, '10000':x3}
        return dict_for_data

    #Примение функции
    dict_for_data = create_dict_for_data(df_for_count_data)

    #Инициализация начального состояния топлива в резервуаре
    def generate_start_volume(azs_name, df_for_count_data):
        matrix_sliv = np.zeros(shape=(len(df_for_count_data), 5), dtype=object)
        #np.random.seed() #стоит добавить если хотим псевдо случайные числа
        start_volume_in_rez = [round(np.random.uniform(0.1, 0.9)*df_for_count_data.loc[0,'rez_volume'],2)\
                          for _ in range(len(df_for_count_data))]
        matrix_sliv[:, 0] = start_volume_in_rez
        return matrix_sliv

    #Применеие функции
    matrix_sliv = generate_start_volume(azs_name, df_for_count_data)
    #print(matrix_sliv)
    # Создание словаря для чеков из 3-ех массивов: пистолете, сливов в литрах и времени слива
    count_check = {0:[[],[],[]], 1:[[],[],[]], 2:[[],[],[]], 3:[[],[],[]], 4:[[],[],[]], 5:[[],[],[]]}
    # Создаем переменную с началом смены
    start_shift = dt.datetime(current_date.year, current_date.month, current_date.day,\
                              df_for_shift_data.loc[0, 'time_start_shift'].hour, 0)\
    - dt.timedelta(days=1) if current_date.hour < 6\
    else dt.datetime(current_date.year, current_date.month, current_date.day, 6, 0)
    # Создание счетчика номеров смен
    shift_num = 1
    # Создание счетчика по количеству изменения номера смен
    count_fact_shift = 1
    # Создание матрицы для записи смен
    matrix_shift = np.array([[shift_num, start_shift, 0],[shift_num, start_shift, 0]], dtype=object)
    # Создание матрицы для записей результата сверки типа 4
    matrix_type_4 = np.zeros(shape=(1, len(df_for_count_data)+2), dtype=object)

    #Функция для создания словарика с состоянием по сливам и остаткам
    def create_matrix_dict():
        matrix_dict = {}
        for i in range(matrix_sliv.shape[0]):
                matrix_dict[i] = matrix_sliv[i,[0,2,3,4]].tolist()
        return matrix_dict
    #Примение функции
    matrix_dict = create_matrix_dict()

    #Инициализация начального состояния времени в чеках
    count_check[0][1].append(current_date)
    count_check[1][1].append(current_date)
    count_check[2][1].append(current_date)
    count_check[3][1].append(current_date)
    count_check[4][1].append(current_date)
    count_check[5][1].append(current_date)

    #Цикл для записи всех дейстрий в матрицу
    while current_date < finish_date: 
        #Инициализация резервуара
        rez_stox = np.random.random_integers(0, len(df_for_count_data) - 1)
        #Инициализация пистолета
        pump_stox = int(df_for_check_data['pump_num'][df_for_check_data['rez_id']==\
                                                      df_for_count_data.loc[rez_stox, 'rez']].sample(n=1))
        #Генерация сливов в литрах
        check_value = round(np.random.normal(30, 10), 2)

        matrix_sliv[rez_stox, 1] = check_value
        count_check[rez_stox][0].append(check_value)
        count_check[rez_stox][2].append(pump_stox)

        parallel_date = count_check[rez_stox][1][-1]\
        + dt.timedelta(seconds=int(np.random.normal(dict_for_data[str(df_for_count_data.loc[0,'rez_volume'])], 10)))
        
        count_check[rez_stox][1].append(parallel_date)

        #Изменение остатков в резервуаре при отпуске клиенту
        matrix_sliv[rez_stox, 0] = matrix_sliv[rez_stox, 0] - matrix_sliv[rez_stox, 1]
        #Обновление даты
        current_date = max([count_check[key][1][-1] for key in count_check.keys() if count_check[key][1]])
        
        # Условие для изменение бригаде по смене
        if current_date > start_shift + dt.timedelta(hours=\
                                                    int(df_for_shift_data.loc[df_for_shift_data[df_for_shift_data['shift_num']==\
                                                                                                shift_num].index[0], 'shift_duration'])):
            # Обновление даты начала смены на новую
            start_shift = dt.datetime(current_date.year, current_date.month, current_date.day, current_date.hour, 0)
            # Запись результата сверки типа 4
            matrix_type_4 = np.vstack((matrix_type_4, np.hstack((matrix_sliv[:, 0], shift_num, start_shift-dt.timedelta(seconds=1)))))
            shift_num += 1
            if shift_num > len(df_for_shift_data.loc[:, 'shift_num']):
                shift_num = 1
            matrix_shift[count_fact_shift, 2] = start_shift - dt.timedelta(seconds=1)
            count_fact_shift += 1
            # Запись факта смены бригады
            matrix_shift = np.vstack((matrix_shift, [shift_num, start_shift, 0]))
            
            

        #Условие для больших резервуаров
        if df_for_count_data.loc[0,'kind_tanker']=='big_tank':
            #Условие для приезда АЦ. Если в одном из резервуаров остатки меньше чем 10% или все резервуары можно долить одним приездом АЦ
            if any(matrix_sliv[:, 0] <= df_for_count_data.loc[0,'rez_volume']*0.1)\
            or ((matrix_sliv[df_for_count_data[df_for_count_data['oil_type']=='ДТ'].index.item(), 0]\
            <(df_for_count_data.loc[0,'rez_volume']\
            -df_tanker.loc['big_tank', 'diesel'])) and (np.count_nonzero(matrix_sliv[:, 0] < (df_for_count_data.loc[0,'rez_volume']\
            -df_tanker.loc['big_tank', 'diesel']) ) >= 4) and (any(matrix_sliv[:, 0] <= (df_for_count_data.loc[0,'rez_volume']\
            -df_tanker.loc['big_tank', 'petrol1'])))):
                #Запись сострояния остатков в переменную
                var = matrix_sliv[:, 0].copy()
                matrix_sliv[df_for_count_data[df_for_count_data['oil_type'] == 'ДТ'].index.item(), 0] = np.inf
                #Цикл который выбирает наименее заполненный резервуар и заполняет его
                for i in range(df_tanker.shape[1]-2):
                    #Заполение столбца матрицы для слива из АЦ
                    matrix_sliv[np.argmin(matrix_sliv[:, 0]), 2] += df_tanker.loc['big_tank', f'petrol{i+1}'] \
                        if (df_for_count_data.loc[0, 'rez_volume'] - matrix_sliv[np.argmin(matrix_sliv[:, 0]), 0]) \
                        > df_tanker.loc['big_tank', f'petrol{i+1}'] else 0
                    #Изменеие текущего времени после первого слива из АЦ
                    matrix_sliv[np.argmin(matrix_sliv[:, 0]), 4] = current_date + dt.timedelta(seconds=int(np.random.normal(2400, 200)))
                    current_date = matrix_sliv[np.argmin(matrix_sliv[:, 0]), 4]
                    #Измение остатков резервуара после каждого слива
                    matrix_sliv[np.argmin(matrix_sliv[:, 0]), 0] += matrix_sliv[np.argmin(matrix_sliv[:, 0]), 2]
                #Возврат состояния резервуара до слива    
                matrix_sliv[:, 0] = var
                #Заполенеие резервуара с Дизелем
                matrix_sliv[df_for_count_data[df_for_count_data['oil_type'] == 'ДТ'].index.item(), 2] = df_tanker.loc['big_tank', 'diesel'] \
                    if (df_for_count_data.loc[0, 'rez_volume'] - matrix_sliv[df_for_count_data[df_for_count_data['oil_type'] == 'ДТ'].index.item(), 0])\
                    > df_tanker.loc['big_tank', 'diesel'] else 0
                #Изменеие текущего времени после первого слива из АЦ
                matrix_sliv[df_for_count_data[df_for_count_data['oil_type'] == 'ДТ'].index.item(), 4] = current_date +\
                dt.timedelta(seconds=int(np.random.normal(2400, 200)))
                current_date = matrix_sliv[df_for_count_data[df_for_count_data['oil_type'] == 'ДТ'].index.item(), 4]

            else:
                #если не подходит под условию слива переходим к следующей иттерации
                continue
        #Условие для малых резервуаров
        else:
            #Условие для приезда АЦ. Если в одном из резервуаров остатки меньше чем 10% или все резервуары можно долить одним приездом АЦ
            if any(matrix_sliv[:, 0] <= df_for_count_data.loc[0,'rez_volume']*0.1)\
            or ((matrix_sliv[df_for_count_data[df_for_count_data['oil_type']=='ДТ'].index.item(), 0]\
            <(df_for_count_data.loc[0,'rez_volume']\
            -df_tanker.loc['small_tank', 'diesel'])) and (np.count_nonzero(matrix_sliv[:, 0] < (df_for_count_data.loc[0,'rez_volume']\
            -df_tanker.loc['small_tank', 'diesel'])) >= 3)):
                #Запись сострояния остатков в переменную
                var = matrix_sliv[:, 0].copy()
                matrix_sliv[df_for_count_data[df_for_count_data['oil_type'] == 'ДТ'].index.item(), 0] = np.inf
                #Цикл который выбирает наименее заполненный резервуар и заполняет его
                for i in range(df_tanker.shape[1]-3):
                    matrix_sliv[np.argmin(matrix_sliv[:, 0]), 2] += df_tanker.loc['small_tank', f'petrol{i+1}'] \
                        if (df_for_count_data.loc[0, 'rez_volume'] - matrix_sliv[np.argmin(matrix_sliv[:, 0]), 0]) \
                        > df_tanker.loc['small_tank', f'petrol{i+1}'] else 0
                    #Изменеие текущего времени после первого слива из АЦ
                    matrix_sliv[np.argmin(matrix_sliv[:, 0]), 4] = current_date + dt.timedelta(seconds=int(np.random.normal(2400, 200)))
                    current_date = matrix_sliv[np.argmin(matrix_sliv[:, 0]), 4]
                    #Измение остатков резервуара после каждого слива
                    matrix_sliv[np.argmin(matrix_sliv[:, 0]), 0] += matrix_sliv[np.argmin(matrix_sliv[:, 0]), 2]
                #Возврат состояния резервуара до слива     
                matrix_sliv[:, 0] = var
                #Заполенеие резервуара с Дизелем
                matrix_sliv[df_for_count_data[df_for_count_data['oil_type'] == 'ДТ'].index.item(), 2] = df_tanker.loc['small_tank', 'diesel'] \
                    if (df_for_count_data.loc[0, 'rez_volume'] - matrix_sliv[df_for_count_data[df_for_count_data['oil_type'] == 'ДТ'].index.item(), 0])\
                    > df_tanker.loc['small_tank', 'diesel'] else 0
                #Изменеие текущего времени после первого слива из АЦ
                matrix_sliv[df_for_count_data[df_for_count_data['oil_type'] == 'ДТ'].index.item(), 4] = current_date +\
                dt.timedelta(seconds=int(np.random.normal(2400, 200)))
                current_date = matrix_sliv[df_for_count_data[df_for_count_data['oil_type'] == 'ДТ'].index.item(), 4]
            else:
                #если не подходит под условию слива переходим к следующей иттерации
                continue
                
        #Записываем результаты после слива из АЦ в столбец матрицы       
        matrix_sliv[:,3] = matrix_sliv[:,0] + matrix_sliv[:,2]
        #Цикл для записи только тех случаев где был слив
        for key, value in matrix_dict.items():
            if 0 in matrix_sliv[key, [0,2,3,4]]:
                continue
            matrix_dict[key] = np.vstack((np.array(value), matrix_sliv[key, [0,2,3,4]])).tolist()
        # Обновление остатков в резервуаре
        matrix_sliv[:,0] = matrix_sliv[:,3]
        
        # Условие для изменение бригаде по смене
        if current_date > start_shift + dt.timedelta(hours=\
                                                    int(df_for_shift_data.loc[df_for_shift_data[df_for_shift_data['shift_num']==\
                                                                                                shift_num].index[0], 'shift_duration'])):
            # Обновление даты начала смены на новую
            start_shift = dt.datetime(current_date.year, current_date.month, current_date.day, current_date.hour, current_date.second)\
            +dt.timedelta(seconds=2)
            # Запись результата сверки типа 4
            matrix_type_4 = np.vstack((matrix_type_4, np.hstack((matrix_sliv[:, 0], shift_num, start_shift- dt.timedelta(seconds=1)))))
            shift_num += 1
            if shift_num > len(df_for_shift_data.loc[:, 'shift_num']):
                shift_num = 1
            matrix_shift[count_fact_shift, 2] = start_shift
            count_fact_shift += 1
            # Запись факта смены бригады
            matrix_shift = np.vstack((matrix_shift, [shift_num, start_shift, 0]))
        
        matrix_sliv[:,2] = 0
        #Инициализация нового состояния времени
        count_check[0][1].append(current_date)
        count_check[1][1].append(current_date)
        count_check[2][1].append(current_date)
        count_check[3][1].append(current_date)
        count_check[4][1].append(current_date)
        count_check[5][1].append(current_date)

    # Запись нового словарика без начального состояния     
    matrix_dict_for_note = {key: value[1:] for key, value in matrix_dict.items()}

    #Функция для удаления из чеков дат по сливам из АЦ
    def clean_data_for_check(count_check):
        for key in count_check.keys():
            count_check[key][1] = [x for x in count_check[key][1] if x not in count_check[5][1]]
        return count_check
    #Применение функции
    dict_for_check = clean_data_for_check(count_check)
    
    # Очистка первого и последнего столбца из расписания смен
    matrix_fact = matrix_shift[1:-1, :]
    # Очистка первого столбца из матрицы сверки номер 4
    matrix_type_4 = matrix_type_4[1:, :]
    
    
    #Функция для формирования таблицы из слованика сливов из АЦ
    def prepare_table_for_note(matrix_dict_for_note, df_for_count_data):
        #Цикл для удаления нулевых значений из словарика
        for key in matrix_dict_for_note.keys():
            matrix_dict_for_note[key] = [elem for elem in matrix_dict_for_note[key] if elem != 0]
        #Формирование дата фрейма с нужными столбцами
        df_for_note = pd.DataFrame(columns=df_for_count_data.columns.values.tolist()[:-2]+['rest', 'fuel', 'actual_weight', 'document_date'])
        i = 0
        #Цикл для записи данных в новый датафрейм из таблицы НСИ и матрицы
        for key in matrix_dict_for_note.keys():
            for value in range(len(matrix_dict_for_note[key])):
                df_for_note.loc[i,'pnpo'] = df_for_count_data.loc[key, 'pnpo']
                df_for_note.loc[i,'azs'] = df_for_count_data.loc[key, 'azs']
                df_for_note.loc[i,'rez_volume'] = df_for_count_data.loc[key, 'rez_volume']
                df_for_note.loc[i,'rez'] = df_for_count_data.loc[key, 'rez']
                df_for_note.loc[i,'oil_id'] = df_for_count_data.loc[key, 'oil_id']
                df_for_note.loc[i,'rest'] = matrix_dict_for_note[key][value][0]
                df_for_note.loc[i,'fuel'] = matrix_dict_for_note[key][value][1]
                df_for_note.loc[i,'actual_weight'] = matrix_dict_for_note[key][value][2]
                df_for_note.loc[i,'document_date'] = matrix_dict_for_note[key][value][3]
                i += 1
        return df_for_note
    #Применение функции
    df_for_note = prepare_table_for_note(matrix_dict_for_note, df_for_count_data)

    #Функция для внесения дебалансов
    def form_debalance(df_for_note, mutual_part_for_noise, positive_part_for_noise,\
                       left_positive_border, right_positive_border, left_negative_border, right_negative_border):
        range_positive_noise = [left_positive_border, right_positive_border]
        range_negative_noise = [left_negative_border, right_negative_border]
        negative_part_for_noise = 1 - positive_part_for_noise
        range_negative_noise = [-1 * val for val in range_negative_noise]
        range_massive = [range_positive_noise, range_negative_noise]
        range_part_noise = [positive_part_for_noise, negative_part_for_noise]
        #Формирование значений которые будут с дебалансов случайным образом
        df_sample = df_for_note.sample(frac=mutual_part_for_noise)
        #Добавление столбца с меткой о том каким будет отклонение
        df_sample['mask_deviation'] = [np.random.choice([0, 1], p=range_part_noise)\
                                      for _ in range(len(df_sample))]
        #Расчет отклонений
        df_sample['actual_weight'] = [df_sample.loc[i,'actual_weight']\
        + df_sample.loc[i,'actual_weight']*np.random.uniform(low=range_massive[0][0],\
                                 high=range_massive[0][1]) if df_sample.loc[i,'mask_deviation']==0
        else df_sample.loc[i,'actual_weight']\
                                            + df_sample.loc[i,'actual_weight']*np.random.uniform(low=range_massive[1][0],\
                                 high=range_massive[1][1]) for i in df_sample.index]
        #Формирование нового столбца и расчет дебалансов в нем
        df_for_note['measured_weight'] = [round(df_sample.loc[i, 'actual_weight'], 2)\
                                          if i in df_sample.index else df_for_note.loc[i, 'actual_weight'] 
                                          for i in df_for_note.index]
        return df_for_note
    #Применение функции
    df_with_debalance = form_debalance(df_for_note, mutual_part_for_noise, positive_part_for_noise,\
                           left_positive_border, right_positive_border, left_negative_border, right_negative_border)

    
    # Функция для подготовки датафрейма по сверке типа 4
    def prepare_df_type_4(matrix_type_4, df_for_count_data):
        i = 0
        df_type_4 = pd.DataFrame()
        for m in range(matrix_type_4.shape[0]):
            for n in range(len(df_for_count_data)):
                df_type_4.loc[i, 'actual_weight'] = matrix_type_4[m, n]
                df_type_4.loc[i, 'shift_num'] = matrix_type_4[m, len(df_for_count_data)]
                df_type_4.loc[i, 'document_date'] = matrix_type_4[m, len(df_for_count_data) + 1]
                df_type_4.loc[i, 'rez_id'] = df_for_count_data.loc[n, 'rez']
                i+=1
        return df_type_4
    df_type_4 = prepare_df_type_4(matrix_type_4, df_for_count_data)
    
    # Функция для подготовки датафрейма по сливам из пистолетов
    def prepare_df_checks(dict_for_check, df_pump):
        lst = []
        for key, values in dict_for_check.items():
            for i in range(len(values[0])):
                lst.append({'volume': values[0][i], 'datetime': values[1][i],\
                            'pump_id' : df_pump.loc[df_pump[(df_pump['azs_id'] == azs_name)\
                                                            & (df_pump['pump_num']==values[2][i])].index[0], 'pump_id']})
        df_checks = pd.DataFrame(lst)
        return df_checks
    df_checks = prepare_df_checks(dict_for_check, df_pump)
    
    # Функция для подготовки датафрейма по графику смен
    def prepare_shift_fact(matrix_fact, df_shift):
        df_shift_fact = pd.DataFrame(matrix_fact, columns=['shift_id', 'start_time', 'end_time'])
        df_shift_fact['shift_id'] = [df_shift['shift_id'][(df_shift['azs_id']==azs_name) \
                 & (df_shift['shift_num']==df_shift_fact.loc[i, 'shift_id'])].values[0] for i in range(len(df_shift_fact))]
        return df_shift_fact
    
    df_shift_fact = prepare_shift_fact(matrix_fact, df_shift)
    
    #Функция для загрузки данных в таблицу
    def load_generate_data_to_bd(df_with_debalance, df_checks, df_type_4, df_shift_fact):
        #Переименование столбцов
        df_for_load_generate_data = df_with_debalance[['pnpo', 'azs', 'rez', 'oil_id',\
                                                       'rest', 'fuel', 'actual_weight', 'document_date', 'measured_weight']]
        df_for_load_generate_data.columns = ['pnpo_id', 'azs_id', 'rez_id', 'oil_id',\
                                            'remaining_fuel', 'draining_fuel', 'actual_weight', 'document_date', 'measured_weight']
        #Смена типа для столбцов с внешними ключами
        df_for_load_generate_data['pnpo_id'] = df_for_load_generate_data['pnpo_id'].astype(int)
        df_for_load_generate_data['azs_id'] = df_for_load_generate_data['azs_id'].astype(int)
        df_for_load_generate_data['rez_id'] = df_for_load_generate_data['rez_id'].astype(int)
        df_for_load_generate_data['oil_id'] = df_for_load_generate_data['oil_id'].astype(int)
        df_for_load_generate_data['remaining_fuel'] = df_for_load_generate_data['remaining_fuel'].astype(float)
        df_for_load_generate_data['draining_fuel'] = df_for_load_generate_data['draining_fuel'].astype(float)
        df_for_load_generate_data['actual_weight'] = df_for_load_generate_data['actual_weight'].astype(float)
        
        # Импорт парметров подключения к БД
        from config import database

        #Подключение к БД
        conn = db.connect(**database)
        cursor = conn.cursor()

        engine = create_engine('postgresql+psycopg2://{user}:{password}@{host}:5432/{database}'.format(**database))

        df_for_load_generate_data.to_sql('cp01_sintetic_data', engine, if_exists='append', index=False, schema='superset')
        df_shift_fact.to_sql('shift_fact', engine, if_exists='append', index=False, schema='superset')
        df_type_4.to_sql('revice4', engine, if_exists='append', index=False, schema='superset')
        df_checks.to_sql('check_goods', engine, if_exists='append', index=False, schema='superset')
        
        conn.commit()
        conn.close()

        return print('По АЗС №', azs_name,'Данные успешно загружены')
    #Применение функции
    finish = load_generate_data_to_bd(df_with_debalance, df_checks, df_type_4, df_shift_fact)

OperationalError: could not translate host name "sph-pgsql01.spherelab.local" to address: Unknown host
